# Smart Lock por reconhecimento de voz




# Bibs necessárias
pip install librosa soundfile numpy sklearn pyaudio, glob

pip install soundfile

In [1]:
import librosa, soundfile, os, glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

Função para extrair as features de mfcc, chroma e mel de um arquivo.
* mfcc: coeficiente cepstral de frequência de Mel, representa o espectro de potência de curto prazo de um som
* chroma: cromagrama de uma forma de onda ou espectrograma de potência.
* mel: frequência do espectrograma Mel


In [2]:
def extrai_atributos(nome_arq):
    X, sample_rate = librosa.load(os.path.join(nome_arq), res_type='kaiser_best')

    stft=np.abs(librosa.stft(X))
    atributos=np.array([])

    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    atributos=np.hstack((atributos, mfccs))

    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    atributos=np.hstack((atributos, chroma))

    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    atributos=np.hstack((atributos, mel))

    return atributos

# Carrega os dados e extrai as features de cada arquivo de áudio
Define os labels para os arquivos de áudio

In [3]:

def load_data(test_size=0.2):
    x,y=[],[]
    for arq in glob.glob('audios/*.wav'):
        nome_arq=os.path.basename(arq)
        atributo=extrai_atributos(arq)
        x.append(atributo)
        
        if nome_arq.startswith('Xabra'):
            y.append('Abra')
        elif nome_arq.startswith('Xfeche'):
            y.append('Fecha')
        else:
            y.append('Desautorizado')
    return train_test_split(np.array(x), y, test_size=test_size, train_size= 0.75, random_state = 11)

# Divide a base de dados
25% p/ teste e 75% p/ treinamento

In [4]:
x_train, x_test,y_train, y_test = load_data(test_size=0.25)

# Quantidade de dados da base de treinamento e de teste:

In [5]:
print((x_train.shape[0], x_test.shape[0]))

(364, 122)


# Número de atributos extraídos

In [6]:
print(f'Atributos extraídos: {x_train.shape[1]}')

Atributos extraídos: 180


# Multi Layer Perceptron

In [7]:
modelo=MLPClassifier(alpha=0.01, batch_size=16, epsilon=1e-08, hidden_layer_sizes=(100,), learning_rate='adaptive', max_iter=500)

# Treinar o modelo


In [8]:
modelo.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=32, hidden_layer_sizes=(75, 50),
              learning_rate='adaptive', max_iter=500)

# Acurácia do modelo
Prediz a saída para a base de testes e depois cálcula a acurácia

In [9]:
y_pred=modelo.predict(x_test)

In [10]:
acuracia=accuracy_score(y_true=y_test, y_pred=y_pred)
print("Acurácia: {:.2f}%".format(acuracia*100))

Acurácia: 96.72%


# Classificação

In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred, target_names=['Abra', 'Desautorizado', 'Fecha']))


               precision    recall  f1-score   support

         Abra       1.00      0.81      0.90        16
Desautorizado       0.96      1.00      0.98        94
        Fecha       1.00      0.92      0.96        12

     accuracy                           0.97       122
    macro avg       0.99      0.91      0.94       122
 weighted avg       0.97      0.97      0.97       122



# Matriz de Confusão

In [12]:
from sklearn.metrics import confusion_matrix
import pandas as pd
cnf_matrix = confusion_matrix(y_test,y_pred)

cnf_table = pd.DataFrame(data=cnf_matrix, index=['Abra', 'Desautorizado', 'Fecha'], columns=["Abra (prev)", "Desautorizado (prev)", "Fecha (prev)"])
print(cnf_table)

               Abra (prev)  Desautorizado (prev)  Fecha (prev)
Abra                    13                     3             0
Desautorizado            0                    94             0
Fecha                    0                     1            11


# Salva o modelo

In [13]:
from joblib import dump

dump(modelo, filename="modelo.joblib")

['modelo.joblib']